**Import Required Libraries**

In [45]:
import tensorflow as tf
import numpy as np
import random
from sklearn.metrics import accuracy_score

**Load Model and Preprocessing Function**

In [46]:
from tensorflow.keras.applications.nasnet import NASNetMobile, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

**Set Constants and Seed**

In [47]:
IMG_SIZE = 224
NUM_CLASSES = 20
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

**Load CIFAR-100 and Select 20 Random Classes**

In [48]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data(label_mode='fine')
y_train = y_train.flatten()
y_test = y_test.flatten()

**Select 20 random classes out of 100**

In [49]:
selected_classes = random.sample(range(100), NUM_CLASSES)
class_map = {original: new for new, original in enumerate(selected_classes)}

**Filter and remap test data**

In [50]:
mask = np.isin(y_test, selected_classes)
x_test = x_test[mask]
y_test = y_test[mask]
y_test = np.vectorize(class_map.get)(y_test)

**Resize and Preprocess Images**

In [51]:
x_test = tf.image.resize(x_test, (IMG_SIZE, IMG_SIZE)).numpy()
x_test = preprocess_input(x_test)

**Create Feature Extractor Model (No Training)**

In [52]:
base_model = NASNetMobile(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Freeze layers

19993432/19993432 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


**Add custom classification head**

In [53]:
x = GlobalAveragePooling2D()(base_model.output)
outputs = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=outputs)

model.summary(show_trainable=True)

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)      ┃ Output Shape    ┃   Param # ┃ Connected to   ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_3     │ (None, 224,     │         0 │ -              │   -   │
│ (InputLayer)      │ 224, 3)         │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ stem_conv1        │ (None, 111,     │       864 │ input_layer_3… │   N   │
│ (Conv2D)          │ 111, 32)        │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ stem_bn1          │ (None, 111,     │       128 │ stem_conv1[0]… │   N   │
│ (BatchNormalizat… │ 111, 32)        │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ activation        │ (None, 111,     │         0 │ stem_bn1[0][0] │   -   │
│ (Activation)      │ 111, 32)        │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ reduction_conv_1… │ (None, 111,     │       352 │ activation[0]… │   N   │
│ (Conv2D)          │ 111, 11)        │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ reduction_bn_1_s… │ (None, 111,     │        44 │ reduction_con… │   N   │
│ (BatchNormalizat… │ 111, 11)        │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ activation_1      │ (None, 111,     │         0 │ reduction_bn_… │   -   │
│ (Activation)      │ 111, 11)        │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ activation_3      │ (None, 111,     │         0 │ stem_bn1[0][0] │   -   │
│ (Activation)      │ 111, 32)        │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ separable_conv_1… │ (None, 115,     │         0 │ activation_1[… │   -   │
│ (ZeroPadding2D)   │ 115, 11)        │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ separable_conv_1… │ (None, 117,     │         0 │ activation_3[… │   -   │
│ (ZeroPadding2D)   │ 117, 32)        │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ separable_conv_1… │ (None, 56, 56,  │       396 │ separable_con… │   N   │
│ (SeparableConv2D) │ 11)             │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ separable_conv_1… │ (None, 56, 56,  │     1,920 │ separable_con… │   N   │
│ (SeparableConv2D) │ 11)             │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ separable_conv_1… │ (None, 56, 56,  │        44 │ separable_con… │   N   │
│ (BatchNormalizat… │ 11)             │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ separable_conv_1… │ (None, 56, 56,  │        44 │ separable_con… │   N   │
│ (BatchNormalizat… │ 11)             │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ activation_2      │ (None, 56, 56,  │         0 │ separable_con… │   -   │
│ (Activation)      │ 11)             │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ activation_4      │ (None, 56, 56,  │         0 │ separable_con… │   -   │
│ (Activation)      │ 11)             │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ separable_conv_2… │ (None, 56, 56,  │       396 │ activation_2[… │   N 

 Total params: 4,290,856 (16.37 MB)

 Trainable params: 21,140 (82.58 KB)

 Non-trainable params: 4,269,716 (16.29 MB)

**Make Predictions and Evaluate**

In [54]:
y_probs = model.predict(x_test, verbose=1)
y_pred = np.argmax(y_probs, axis=1)

accuracy = accuracy_score(y_test, y_pred)
print(f"✅ NASNetMobile Feature Extraction Accuracy: {accuracy:.4f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 33s 286ms/step
✅ NASNetMobile Feature Extraction Accuracy: 0.0390
